In [54]:
import cv2
import math

In [55]:
import numpy as np
import time
import dlib
from scipy.spatial import distance as dist

In [56]:
def eye_aspect_ratio(eye):
    # compute the euclidean distance between the vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye landmarks
    C = dist.euclidean(eye[0], eye[3])

    # compute the EAR
    ear = (A + B) / (2 * C)
    return ear

# to detect the facial region
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/mini/shape_predictor_68_face_landmarks.dat')

In [57]:
from flask import Flask, render_template, Response,jsonify
app = Flask(__name__)



cap=cv2.VideoCapture(0) #0 for 1st webcam
font = cv2.FONT_HERSHEY_PLAIN
starting_time= time.time()

def gen_frames():
    JAWLINE_POINTS = list(range(0, 17))
    RIGHT_EYEBROW_POINTS = list(range(17, 22))
    LEFT_EYEBROW_POINTS = list(range(22, 27))
    NOSE_POINTS = list(range(27, 36))
    RIGHT_EYE_POINTS = list(range(36, 42))
    LEFT_EYE_POINTS = list(range(42, 48))
    MOUTH_OUTLINE_POINTS = list(range(48, 61))
    MOUTH_INNER_POINTS = list(range(61, 68))

    EYE_AR_THRESH = 0.22
    EYE_AR_CONSEC_FRAMES = 3
    EAR_AVG = 0

    COUNTER = 0
    TOTAL = 0
    
    
    while True:
        _,frame= cap.read() # 
        #frame_id+=1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        for rect in rects:
            x = rect.left()
            y = rect.top()
            x1 = rect.right()
            y1 = rect.bottom()
            # get the facial landmarks
            landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])
            # get the left eye landmarks
            left_eye = landmarks[LEFT_EYE_POINTS]
            # get the right eye landmarks
            right_eye = landmarks[RIGHT_EYE_POINTS]
            # draw contours on the eyes
            left_eye_hull = cv2.convexHull(left_eye)
            right_eye_hull = cv2.convexHull(right_eye)
            #cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1) # (image, [contour], all_contours, color, thickness)
            #cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
            # compute the EAR for the left eye
            ear_left = eye_aspect_ratio(left_eye)
            # compute the EAR for the right eye
            ear_right = eye_aspect_ratio(right_eye)
            # compute the average EAR
            ear_avg = (ear_left + ear_right) / 2.0
            # detect the eye blink
            if ear_avg < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1
                    #print("Eye blinked")
                COUNTER = 0
            if TOTAL == 1:
                cv2.putText(frame,"Avocado",(30,60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
            elif TOTAL == 2:
                cv2.putText(frame,"Avocado",(30,60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Sweet Potatao",(30,90),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
            elif TOTAL ==3:
                cv2.putText(frame,"Avocado",(30,60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Sweet Potatao",(30,90),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Rice",(30,120),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
            elif TOTAL ==4:
                cv2.putText(frame,"Avocado",(30,60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Sweet Potatao",(30,90),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Rice",(30,120),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Banana",(30,150),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
            elif TOTAL ==5:
                cv2.putText(frame,"Avocado",(30,60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Sweet Potatao",(30,90),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Rice",(30,120),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Banana",(30,150),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
                cv2.putText(frame,"Date",(30,180),cv2.FONT_HERSHEY_DUPLEX,0.7,(0,255,255))
            cv2.putText(frame, "Blinks{}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            #cv2.putText(frame, "EAR {}".format(ear_avg), (10, 60), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
    
        height,width,channels = frame.shape
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


In [58]:
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

In [59]:
@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('video.html')


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2022 11:18:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 11:18:41] "GET /video_feed HTTP/1.1" 200 -


In [60]:
cap.release()

Error on request:
Traceback (most recent call last):
  File "/home/mini/.local/lib/python3.6/site-packages/werkzeug/serving.py", line 323, in run_wsgi
    execute(self.server.app)
  File "/home/mini/.local/lib/python3.6/site-packages/werkzeug/serving.py", line 314, in execute
    for data in application_iter:
  File "/home/mini/.local/lib/python3.6/site-packages/werkzeug/wsgi.py", line 506, in __next__
    return self._next()
  File "/home/mini/.local/lib/python3.6/site-packages/werkzeug/wrappers/base_response.py", line 45, in _iter_encoded
    for item in iterable:
  File "<ipython-input-57-3dc2fdfc7529>", line 31, in gen_frames
    
cv2.error: OpenCV(3.4.2) /io/opencv/modules/imgproc/src/color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'CvtHelper'


In [45]:
img=cv2.imread('vata.png')
img=cv2.resize(img,(480,640))
img_g=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #converting to gray scale aS threshold functions for grey image
ret,mask= cv2.threshold(img_g,10,255,cv2.THRESH_BINARY) # if thr value is less than 10=> 0, otherwise 255
mask_i=cv2.bitwise_not(mask)
cv2.imshow("t",mask)
cv2.imshow("1m",mask_i)

In [18]:
img.shape


(394, 470, 3)

In [46]:
cv2.destroyAllWindows()